In [1]:
#加载预训练的Inception V3模型
from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0) #禁用与训练相关的操作
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
#设置DeepDream配置
layer_contributions = {'mixed2':0.2, 'mixed3':3., 'mixed4':2., 'mixed5':1.5}

In [3]:
#定义需要最大化的损失
layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [4]:
#梯度上升过程
dream = model.input

grads = K.gradients(loss, dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

In [5]:
#辅助函数
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1, float(size[0]) / img.shape[1], float(size[1]) / img.shape[2], 1)
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)
    
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *=255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
#在多个连续尺度上运行梯度上升
import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10

base_image_path = 'D:/Documents/Pictures/pictures/bread.jpg'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
    
save_img(img, fname='final_dream.png')

Processing image shape (326, 522)
...Loss value at 0 : 1.8613449
...Loss value at 1 : 2.5828028
...Loss value at 2 : 3.4053845
...Loss value at 3 : 4.253265
...Loss value at 4 : 5.0418043
...Loss value at 5 : 5.762251
...Loss value at 6 : 6.44879
...Loss value at 7 : 7.070774
...Loss value at 8 : 7.7562366
...Loss value at 9 : 8.359261
...Loss value at 10 : 8.948978
...Loss value at 11 : 9.521598


C:\Users\Administrator\Anaconda3\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  if sys.path[0] == '':
C:\Users\Administrator\Anaconda3\envs\Tensorflow\lib\site-packages\scipy\ndimage\interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


Processing image shape (457, 731)
...Loss value at 0 : 3.1653442
...Loss value at 1 : 4.6054716
...Loss value at 2 : 5.779333
...Loss value at 3 : 6.7757206
...Loss value at 4 : 7.689292
...Loss value at 5 : 8.535756
...Loss value at 6 : 9.286682
...Loss value at 7 : 9.994608
Processing image shape (640, 1024)
...Loss value at 0 : 3.3502498
...Loss value at 1 : 4.7701116
